In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08701348304748535
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 30.35443578868086
200000 53.63104865332716
300000 57.98395573030578
400000 53.61175089832416
500000 53.09120844539382
600000 52.382154537326095
700000 59.86075463326993
800000 65.15947171973767
900000 72.41238363021348
1000000 74.14415884769853
1100000 77.51735649439132
1200000 86.79613663253319
1300000 29.95672166080468
1400000 100.71934077178211
1500000 110.28489052472825
1600000 109.39318724508549
1700000 108.44064265277844
1800000 108.4081045358386
1900000 105.69915340673413
2000000 106.46645930020645
2100000 100.65773951277103
2200000 100.51040241319356
2300000 109.42474203823872
2400000 101.01314482040274
2500000 100.8293600498617
2600000 33.004625579719104
2700000 107.39022704446964
2800000 96.98343367760268
2900000 98.42379428446185
3000000 96.15

29400000 108.85257889611349
29500000 104.39384897007305
29600000 107.27770304319091
29700000 108.24594852311402
29800000 107.09736908290407
29900000 109.28109305580645
30000000 70.44800930829284
30100000 86.9172009420944
30200000 113.09857438992721
30300000 103.82501553573621
30400000 89.7631412377377
30500000 60.48138522869605
30600000 106.90914757008865
30700000 105.23382317357557
30800000 104.03990580383822
30900000 103.47473588917128
31000000 109.7366430412381
31100000 107.06287531255833
31200000 97.96326522979274
31300000 105.88030512102671
31400000 100.17429828436929
31500000 21.195242150381926
31600000 103.51221349934175
31700000 104.34550690474309
31800000 97.73100976527883
31900000 111.70700009238242
32000000 107.43859776838013
32100000 102.21736650592474
32200000 106.57824402551431
32300000 108.3365076823506
32400000 105.33752486506923
32500000 62.518533010462214
32600000 89.52838103726815
32700000 113.00856299351862
32800000 107.63159677224874
32900000 113.06773123388417
330

59200000 43.81253251736692
59300000 97.45143709322339
59400000 94.82309650950525
59500000 97.71829571179634
59600000 99.21072201681018
59700000 101.43991547914486
59800000 102.05533837452339
59900000 101.19390580528578
60000000 35.82695098160574
60100000 114.17052861250811
60200000 109.72342255672405
60300000 104.58871426267466
60400000 113.36358711880425
60500000 105.32223501408599
60600000 99.22874449439078
60700000 105.75774675178293
60800000 91.43095964247043
60900000 50.51575470252846
61000000 100.99556960736855
61100000 96.72623137203323
61200000 103.7221167413851
61300000 101.94976011272585
61400000 94.15242722849042
61500000 90.91396731469324
61600000 90.3594057356911
61700000 28.263746935570445
61800000 99.17154585582284
61900000 94.22594359243269
62000000 97.42382118269812
62100000 94.24805993780292
62200000 94.10815199309849
62300000 100.38689013036567
62400000 102.40802466262564
62500000 36.54140570332387
62600000 106.27467314934115
62700000 102.83362441244567
62800000 99.7

89400000 54.11338052488381
89500000 89.85636125059281
89600000 93.58394669320643
89700000 91.83491720483285
89800000 89.12767458241372
89900000 74.33307133576797
90000000 57.480278440874756
90100000 96.26985378131282
90200000 96.89087646095797
90300000 95.51261729714344
90400000 97.85972007820541
90500000 103.45392332232984
90600000 96.96982814038185
90700000 96.93121707394853
90800000 98.79987208317618
90900000 96.77280485664889
91000000 92.72578861962968
91100000 100.79147655926697
91200000 88.26221095357126
91300000 39.58290546320087
91400000 89.18894162917923
91500000 89.22413462341424
91600000 92.38042515918023
91700000 86.14897360593525
91800000 76.23449170275046
91900000 41.30500681919848
92000000 89.26071513846813
92100000 94.68621065284238
92200000 89.17063977777697
92300000 86.7742845891645
92400000 70.44455786279867
92500000 58.77049119531968
92600000 93.77137467358124
92700000 93.03964747926197
92800000 95.28453300765698
92900000 98.76815057523616
93000000 101.0879861135671

119000000 85.26593224686897
119100000 64.07500200034583
119200000 61.64977343141947
119300000 88.1717711152452
119400000 93.99273190697232
119500000 50.03040841206038
119600000 67.4963488070892
119700000 84.24673786497107
119800000 90.7144651208029
119900000 50.51235670580365
120000000 76.63122043263962
120100000 98.46727766738745
120200000 89.23269085169382
120300000 91.51672522094826
120400000 97.82001125517048
120500000 99.70570988849329
120600000 98.30562383725858
120700000 99.9735701282876
120800000 102.10491960301387
120900000 95.94796764391742
121000000 94.80672207964857
121100000 92.2994106209933
121200000 91.31734520028263
121300000 89.61749796925973
121400000 90.61928279602019
121500000 86.96691233011165
121600000 86.08103057864906
121700000 33.86145415873777
121800000 90.1777012129879
121900000 90.0505077270277
122000000 64.48620599273241
122100000 53.990361105147954
122200000 85.24052155010054
122300000 105.81190425893378
122400000 45.82782562634875
122500000 78.05283399986

...

953000000 1.596719134024598
953100000 1.7057917826711873
953200000 1.5128047481707771
953300000 1.8527630626880218
953400000 0.77038698312729
953500000 0.8891748307512151
953600000 1.8947386451115966
953700000 1.7970340406386016
953800000 1.5576173453305748
953900000 1.5125776839040432
954000000 1.088087375598808
954100000 0.6101653307808664
954200000 2.0797879393901857
954300000 1.5937528653327944
954400000 1.6883786243226528
954500000 1.5862684055153402
954600000 1.5582663020496388
954700000 0.5532572903279557
954800000 1.3951253487089053
954900000 1.6090526407127996
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 2.0081983063526874
955100000 1.5894703390496507
955200000 1.4307173146083354
955300000 1.2896460841846664
955400000 1.2319642653282186
955500000 1.5192385134881696
955600000 1.1893317122593192
955700000 1.7233138658917335
955800000 1.6596765963061513
955900000 1.188492350101206
956000000 1.3465059173196845
956100000 1.168050054711473
95620000

best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.3489818778554956
979300000 0.3442256769385668
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.32051034352624747
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3125190815098643
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.6874352126885461
979700000 1.851447868541426
979800000 1.7348930324359935
979900000 1.40038991776827
980000000 1.4344068899414395
980100000 1.66415008356147
980200000 1.4901373786015346
980300000 1.4396827861414523
980400000 1.2915234339398503
980500000 1.4586088984662948
980600000 1.2839329807243136
980700000 1.3576632868864955
980800000 1.6408125857753044
980900000 1.2266757695442723
9810

998500000 1.203792123718157
998600000 1.349817650188266
998700000 0.9201395410842876
998800000 1.137900192800135
998900000 1.3399122289693532
999000000 0.8942910190180242
999100000 1.1131189495263683
999200000 1.3234692044512617
999300000 1.0089104196879148
999400000 1.0035488587894459
999500000 1.2983677842920553
999600000 1.0967999161231894
999700000 0.833481133639282
999800000 1.2825024420896505
999900000 1.4151960652520277
1000000000 0.9092877042502934
1000100000 1.3229636725996257
1000200000 1.3576412177529216
1000300000 0.9085390901823355
1000400000 1.3786708066438635
1000500000 0.241683033463039
1000600000 0.2458627063414474
1000700000 0.24248350660160314
1000800000 0.26462428283982276
1000900000 0.24109525521270964
1001000000 0.24070800333648257
1001100000 0.2423818571989715
1001200000 0.24369261151152719
1001300000 0.2367404514481028
1001400000 0.24761762673533166
1001500000 0.26698488128548525
1001600000 0.2414363399666323
1001700000 0.23849801571128768
1001800000 0.237941929

1025400000 0.251218681856221
1025500000 0.27104971741595535
1025600000 0.35883805398535734
1025700000 0.2669062933717708
1025800000 0.24196693944491612
1025900000 0.3365611511904677
1026000000 0.2967301663871666
1026100000 0.20896125316988495
1026200000 0.34238154497557416
1026300000 0.3022392005796731
1026400000 0.1897966102168772
1026500000 0.34290882996833594
1026600000 0.31300036734783054
1026700000 0.17112011471527352
1026800000 0.3392390434241831
1026900000 0.3020404354047517
1027000000 0.1996306428994669
1027100000 0.31255448812141884
1027200000 0.3183326648830712
1027300000 0.2210686781617138
1027400000 0.28396131501002314
1027500000 0.32672942071829236
1027600000 0.24240063552788169
1027700000 0.3001029195509712
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.3281097126211464
1027900000 0.2570622867706676
1028000000 0.23630090719174146
1028100000 0.2575388824782491
1028200000 0.258759421735894
1028300000 0.22312516155359677
1028400000 0.24648

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.044537473586675196
1045500000 0.05121695083461205
1045600000 0.04623535613061984
1045700000 0.04069055479630828
1045800000 0.04523818704198201
1045900000 0.035747341921962626
1046000000 0.02759927035001715
1046100000 0.010199158378046752
1046200000 0.029954403925308258
1046300000 0.029823948681177697
1046400000 0.033266189535971484
1046500000 0.028224061787875492
1046600000 0.02667684962733653
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.027829775684684515
1046800000 0.014712122743618487
1046900000 0.013751268210560748
1047000000 0.010684596102517843
1047100000 0.005158756615316867
1047200000 0.008584752325859335
1047300000 0.009256702575403453
best so far: 0
type: [4, 1, 1,